In [4]:
! pip install opencv-python


  Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_x86_64.whl.metadata (19 kB)
Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_x86_64.whl (57.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 273.3 kB/s  0:00:23m0:00:0100:02
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [opencv-python]0m [opencv-python]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.16.2 requires numpy<2.0.0,>=1.26.0; python_version >= "3.12", but you have numpy 2.2.6 which is incompatible.


In [6]:
! pip uninstall -y numpy scipy seaborn tensorflow opencv-python


Found existing installation: numpy 2.2.6
Uninstalling numpy-2.2.6:
  Successfully uninstalled numpy-2.2.6
Found existing installation: scipy 1.16.3
Uninstalling scipy-1.16.3:
  Successfully uninstalled scipy-1.16.3
Found existing installation: seaborn 0.13.2
Uninstalling seaborn-0.13.2:
  Successfully uninstalled seaborn-0.13.2
Found existing installation: tensorflow 2.16.2
Uninstalling tensorflow-2.16.2:
  Successfully uninstalled tensorflow-2.16.2
Found existing installation: opencv-python 4.12.0.88
Uninstalling opencv-python-4.12.0.88:
  Successfully uninstalled opencv-python-4.12.0.88


In [7]:
! pip install \
numpy==1.26.4 \
scipy==1.11.4 \
pandas==2.2.2 \
matplotlib==3.8.4 \
seaborn==0.13.2 \
opencv-python==4.9.0.80 \
tensorflow==2.16.2


  Using cached numpy-1.26.4-cp312-cp312-macosx_10_9_x86_64.whl.metadata (61 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached tensorflow-2.16.2-cp312-cp312-macosx_10_15_x86_64.whl.metadata (4.1 kB)
Using cached numpy-1.26.4-cp312-cp312-macosx_10_9_x86_64.whl (20.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.1/37.1 MB 1.9 MB/s  0:00:19m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 2.7 MB/s  0:00:04m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 1.8 MB/s  0:00:04m0:00:0100:01
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 MB 2.4 MB/s  0:00:22m0:00:0100:01
Using cached tensorflow-2.16.2-cp312-cp312-macosx_10_15_x86_64.whl (259.7 MB)
  Attempting uninstall: pandas0m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/7 [scipy]
    Found existing installation: pandas 2.3.3━━━━━━━━━━━━━━━━━ 1/7 [scipy]
    Uninstalling pandas-2.3.3:━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/

In [2]:

import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, SeparableConv2D, MaxPooling2D, Lambda
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose, Conv2D, add, concatenate
from tensorflow.keras.layers import LeakyReLU, Activation, Reshape
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau

2025-12-26 20:34:41.463118: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
attributes=pd.read_csv('list_attr_celeba.csv')
bboxes=pd.read_csv('list_bbox_celeba.csv')
partition=pd.read_csv('list_eval_partition.csv')
landmarks=pd.read_csv('list_landmarks_align_celeba.csv')

In [8]:
from pathlib import Path

# Resolve the CSV relative to the notebook location
base_dir = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()
landmarks_path = base_dir / 'list_landmarks_align_celeba.csv'

if not landmarks_path.exists():
    raise FileNotFoundError(f"Missing file: {landmarks_path}")

landmarks_preview = pd.read_csv(landmarks_path)
landmarks_preview.head()

,image_id,lefteye_x,lefteye_y,righteye_x,righteye_y,nose_x,nose_y,leftmouth_x,leftmouth_y,rightmouth_x,rightmouth_y
0,000001.jpg,69,109,106,113,77,142,73,152,108,154
1,000002.jpg,69,110,107,112,81,135,70,151,108,153
2,000003.jpg,76,112,104,106,108,128,74,156,98,158
3,000004.jpg,72,113,108,108,101,138,71,155,101,151
4,000005.jpg,66,114,112,112,86,119,71,147,104,150


In [12]:
from pathlib import Path
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

base_dir = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()
candidate_dirs = [base_dir / 'img_align_celeba', base_dir / 'data' / 'img_align_celeba']
img_files = []
for folder in candidate_dirs:
    if folder.exists():
        img_files = sorted(folder.glob('*.jpg'))
        if img_files:
            break

if not img_files:
    print('No images found. Place sample JPGs in img_align_celeba/ or data/img_align_celeba/.')
else:
    images = img_files[:20]
    cols = 5
    rows = math.ceil(len(images) / cols)
    plt.figure(figsize=(cols * 3, rows * 3))
    for idx, img_path in enumerate(images):
        plt.subplot(rows, cols, idx + 1)
        plt.axis('off')
        plt.title(img_path.name, fontsize=8)
        plt.imshow(mpimg.imread(img_path))
    plt.tight_layout()

No images found. Place sample JPGs in img_align_celeba/ or data/img_align_celeba/.


In [13]:
landmarks_df=pd.read_csv('list_landmarks_align_celeba.csv')
landmarks_df.head()

,image_id,lefteye_x,lefteye_y,righteye_x,righteye_y,nose_x,nose_y,leftmouth_x,leftmouth_y,rightmouth_x,rightmouth_y
0,000001.jpg,69,109,106,113,77,142,73,152,108,154
1,000002.jpg,69,110,107,112,81,135,70,151,108,153
2,000003.jpg,76,112,104,106,108,128,74,156,98,158
3,000004.jpg,72,113,108,108,101,138,71,155,101,151
4,000005.jpg,66,114,112,112,86,119,71,147,104,150


In [16]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Locate data (include nested img_align_celeba/img_align_celeba case)
base_dir = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()
extra_root = Path.home() / 'Downloads' / 'archive (5)' / 'img_align_celeba'
extra_nested = extra_root / 'img_align_celeba'
img_dir_candidates = [
    base_dir / 'img_align_celeba',
    base_dir / 'data' / 'img_align_celeba',
    extra_root,
    extra_nested,
]
img_dir = next((d for d in img_dir_candidates if d.exists()), None)
landmarks_path = base_dir / 'list_landmarks_align_celeba.csv'

print(f"Checking image dirs (exists?): {[str(d) + ' ✔' if d.exists() else str(d) + ' ✖' for d in img_dir_candidates]}")

if img_dir is None:
    print('No image directory found. Create img_align_celeba/ or data/img_align_celeba/ or update extra path.')
elif not landmarks_path.exists():
    print(f'Missing landmarks file: {landmarks_path}')
else:
    landmarks = pd.read_csv(landmarks_path)
    img_path = None
    landmark_row = None
    for _, row in landmarks.iterrows():
        candidate = img_dir / row['image_id']
        if candidate.exists():
            img_path = candidate
            landmark_row = row
            break

    if img_path is None:
        print(f'No matching image files found in {img_dir} for entries in landmarks CSV.')
    else:
        img = mpimg.imread(img_path)
        plt.figure(figsize=(6, 6))
        plt.imshow(img)
        keypoints = ['lefteye', 'righteye', 'nose', 'leftmouth', 'rightmouth']
        for key in keypoints:
            x = landmark_row[f'{key}_x']
            y = landmark_row[f'{key}_y']
            plt.scatter(x, y, c='red', s=30)
            plt.text(x + 2, y - 2, key, color='yellow', fontsize=8)
        plt.title(f'Landmarks for {img_path.name}')
        plt.axis('off')
        plt.tight_layout()

Checking image dirs (exists?): ['/Users/imac/DeepLearning2/img_align_celeba ✖', '/Users/imac/DeepLearning2/data/img_align_celeba ✖', '/Users/imac/Downloads/archive (5)/img_align_celeba ✔', '/Users/imac/Downloads/archive (5)/img_align_celeba/img_align_celeba ✔']
No matching image files found in /Users/imac/Downloads/archive (5)/img_align_celeba for entries in landmarks CSV.


In [17]:
bboxes_df=pd.read_csv('list_bbox_celeba.csv')
bboxes_df.head()

,image_id,x_1,y_1,width,height
0,000001.jpg,95,71,226,313
1,000002.jpg,72,94,221,306
2,000003.jpg,216,59,91,126
3,000004.jpg,622,257,564,781
4,000005.jpg,236,109,120,166


In [19]:
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Plot a bounding box using existing bboxes_df
globals_present = globals()
if 'bboxes_df' not in globals_present:
    print('bboxes_df not defined. Run the cell that loads list_bbox_celeba.csv first.')
else:
    base_dir = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()
    img_candidates = [
        base_dir / 'img_align_celeba',
        base_dir / 'data' / 'img_align_celeba',
        Path.home() / 'Downloads' / 'archive (5)' / 'img_align_celeba',
        Path.home() / 'Downloads' / 'archive (5)' / 'img_align_celeba' / 'img_align_celeba',
    ]
    img_dir = next((d for d in img_candidates if d.exists()), None)
    print('Image dirs:', [f"{d}{' ✔' if d.exists() else ' ✖'}" for d in img_candidates])

    if img_dir is None:
        print('No image directory found. Update img_candidates to your path.')
    else:
        img_path = None
        bbox_row = None
        for _, row in bboxes_df.iterrows():
            candidate = img_dir / row['image_id']
            if candidate.exists():
                img_path = candidate
                bbox_row = row
                break

        if img_path is None:
            print(f'No matching image files found in {img_dir} for entries in bboxes_df.')
        else:
            img = mpimg.imread(img_path)
            fig, ax = plt.subplots(figsize=(6, 6))
            ax.imshow(img)
            rect = plt.Rectangle(
                (bbox_row['x_1'], bbox_row['y_1']),
                bbox_row['width'],
                bbox_row['height'],
                linewidth=2,
                edgecolor='cyan',
                facecolor='none'
            )
            ax.add_patch(rect)
            ax.set_title(f"Bounding box for {img_path.name}")
            ax.axis('off')
            plt.tight_layout()

Image dirs: ['/Users/imac/DeepLearning2/img_align_celeba ✖', '/Users/imac/DeepLearning2/data/img_align_celeba ✖', '/Users/imac/Downloads/archive (5)/img_align_celeba ✔', '/Users/imac/Downloads/archive (5)/img_align_celeba/img_align_celeba ✔']
No matching image files found in /Users/imac/Downloads/archive (5)/img_align_celeba for entries in bboxes_df.
